## ДЗ 1 (ОБЯЗАТЕЛЬНОЕ): Анализ температурных данных и мониторинг текущей температуры через OpenWeatherMap API

**Описание задания:**  
Вы аналитик в компании, занимающейся изучением климатических изменений и мониторингом температур в разных городах. Вам нужно провести анализ исторических данных о температуре для выявления сезонных закономерностей и аномалий. Также необходимо подключить API OpenWeatherMap для получения текущей температуры в выбранных городах и сравнить её с историческими данными.


### Цели задания:
1. Провести **анализ временных рядов**, включая:
   - Вычисление скользящего среднего и стандартного отклонения для сглаживания температурных колебаний.
   - Определение аномалий на основе отклонений температуры от $ \text{скользящее среднее} \pm 2\sigma $.
   - Построение долгосрочных трендов изменения температуры.
   - Любые дополнительные исследования будут вам в плюс.

2. Осуществить **мониторинг текущей температуры**:
   - Получить текущую температуру через OpenWeatherMap API.
   - Сравнить её с историческим нормальным диапазоном для текущего сезона.

3. Разработать **интерактивное приложение**:
   - Дать пользователю возможность выбрать город.
   - Отобразить результаты анализа температур, включая временные ряды, сезонные профили и аномалии.
   - Провести анализ текущей температуры в контексте исторических данных.


### Описание данных
Исторические данные о температуре содержатся в файле `temperature_data.csv`, включают:
  - `city`: Название города.
  - `timestamp`: Дата (с шагом в 1 день).
  - `temperature`: Среднесуточная температура (в °C).
  - `season`: Сезон года (зима, весна, лето, осень).

Код для генерации файла вы найдете ниже.

### Этапы выполнения

1. **Анализ исторических данных**:
   - Вычислить **скользящее среднее** температуры с окном в 30 дней для сглаживания краткосрочных колебаний.
   - Рассчитать среднюю температуру и стандартное отклонение для каждого сезона в каждом городе.
   - Выявить аномалии, где температура выходит за пределы $ \text{среднее} \pm 2\sigma $.
   - Попробуйте распараллелить проведение этого анализа. Сравните скорость выполнения анализа с распараллеливанием и без него.

2. **Мониторинг текущей температуры**:
   - Подключить OpenWeatherMap API для получения текущей температуры города. Для получения API Key (бесплатно) надо зарегистрироваться на сайте. Обратите внимание, что API Key может активироваться только через 2-3 часа, это нормально. Посему получите ключ заранее.
   - Получить текущую температуру для выбранного города через OpenWeatherMap API.
   - Определить, является ли текущая температура нормальной, исходя из исторических данных для текущего сезона.
   - Данные на самом деле не совсем реальные (сюрпрайз). Поэтому на момент эксперимента погода в Берлине, Каире и Дубае была в рамках нормы, а в Пекине и Москве аномальная. Протестируйте свое решение для разных городов.
   - Попробуйте для получения текущей температуры использовать синхронные и асинхронные методы. Что здесь лучше использовать?

3. **Создание приложения на Streamlit**:
   - Добавить интерфейс для загрузки файла с историческими данными.
   - Добавить интерфейс для выбора города (из выпадающего списка).
   - Добавить форму для ввода API-ключа OpenWeatherMap. Когда он не введен, данные для текущей погоды не показываются. Если ключ некорректный, выведите на экран ошибку (должно приходить `{"cod":401, "message": "Invalid API key. Please see https://openweathermap.org/faq#error401 for more info."}`).
   - Отобразить:
     - Описательную статистику по историческим данным для города, можно добавить визуализации.
     - Временной ряд температур с выделением аномалий (например, точками другого цвета).
     - Сезонные профили с указанием среднего и стандартного отклонения.
   - Вывести текущую температуру через API и указать, нормальна ли она для сезона.

### Критерии оценивания

- Корректное проведение анализа данных – 1 балл.
- Исследование распараллеливания анализа – 1 балл.
- Корректный поиск аномалий – 1 балл.
- Подключение к API и корректность выполнения запроса – 1 балл.
- Проведение эксперимента с синхронным и асинхронным способом запроса к API – 1 балл.
- Создание интерфейса приложения streamlit в соответствии с описанием – 3 балла.
- Корректное отображение графиков и статистик, а также сезонных профилей – 1 балл.
- Корректный вывод текущей температуры в выбранном городе и проведение проверки на ее аномальность – 1 балл.
- Любая дополнительная функциональность приветствуется и оценивается бонусными баллами (не более 2 в сумме) на усмотрение проверяющего.

### Формат сдачи домашнего задания

Решение нужно развернуть в Streamlit Cloud (бесплатно)

*   Создаем новый репозиторий на GitHub.  
*   Загружаем проект.
*   Создаем аккаунт в [Streamlit Cloud](https://streamlit.io/cloud).
*   Авторизуемся в Streamlit Cloud.
*   Создаем новое приложение в Streamlit Cloud и подключаем GitHub-репозиторий.
*   Deploy!

Сдать в форму необходимо:
1. Ссылку на развернутое в Streamlit Cloud приложение.
2. Ссылку на код. Все выводы про, например, использование параллельности/асинхронности опишите в комментариях.

Не забудьте удалить ключ API и иную чувствительную информацию.

In [1]:
import pandas as pd
import numpy as np

# Реальные средние температуры (примерные данные) для городов по сезонам
seasonal_temperatures = {
    "New York": {"winter": 0, "spring": 10, "summer": 25, "autumn": 15},
    "London": {"winter": 5, "spring": 11, "summer": 18, "autumn": 12},
    "Paris": {"winter": 4, "spring": 12, "summer": 20, "autumn": 13},
    "Tokyo": {"winter": 6, "spring": 15, "summer": 27, "autumn": 18},
    "Moscow": {"winter": -10, "spring": 5, "summer": 18, "autumn": 8},
    "Sydney": {"winter": 12, "spring": 18, "summer": 25, "autumn": 20},
    "Berlin": {"winter": 0, "spring": 10, "summer": 20, "autumn": 11},
    "Beijing": {"winter": -2, "spring": 13, "summer": 27, "autumn": 16},
    "Rio de Janeiro": {"winter": 20, "spring": 25, "summer": 30, "autumn": 25},
    "Dubai": {"winter": 20, "spring": 30, "summer": 40, "autumn": 30},
    "Los Angeles": {"winter": 15, "spring": 18, "summer": 25, "autumn": 20},
    "Singapore": {"winter": 27, "spring": 28, "summer": 28, "autumn": 27},
    "Mumbai": {"winter": 25, "spring": 30, "summer": 35, "autumn": 30},
    "Cairo": {"winter": 15, "spring": 25, "summer": 35, "autumn": 25},
    "Mexico City": {"winter": 12, "spring": 18, "summer": 20, "autumn": 15},
}

# Сопоставление месяцев с сезонами
month_to_season = {12: "winter", 1: "winter", 2: "winter",
                   3: "spring", 4: "spring", 5: "spring",
                   6: "summer", 7: "summer", 8: "summer",
                   9: "autumn", 10: "autumn", 11: "autumn"}

# Генерация данных о температуре
def generate_realistic_temperature_data(cities, num_years=10):
    dates = pd.date_range(start="2010-01-01", periods=365 * num_years, freq="D")
    data = []

    for city in cities:
        for date in dates:
            season = month_to_season[date.month]
            mean_temp = seasonal_temperatures[city][season]
            # Добавляем случайное отклонение
            temperature = np.random.normal(loc=mean_temp, scale=5)
            data.append({"city": city, "timestamp": date, "temperature": temperature})

    df = pd.DataFrame(data)
    df['season'] = df['timestamp'].dt.month.map(lambda x: month_to_season[x])
    return df

# Генерация данных
data = generate_realistic_temperature_data(list(seasonal_temperatures.keys()))
data.to_csv('temperature_data.csv', index=False)


In [2]:
# Загружаем сгенерированные данные

df = pd.read_csv('temperature_data.csv')
df

,city,timestamp,temperature,season
0,New York,2010-01-01,-7.000078,winter
1,New York,2010-01-02,0.251163,winter
2,New York,2010-01-03,-5.185995,winter
3,New York,2010-01-04,5.994464,winter
4,New York,2010-01-05,-1.887698,winter
...,...,...,...,...
54745,Mexico City,2019-12-25,13.450151,winter
54746,Mexico City,2019-12-26,7.375380,winter
54747,Mexico City,2019-12-27,8.762830,winter
54748,Mexico City,2019-12-28,15.371163,winter


In [3]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54750 entries, 0 to 54749
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   city         54750 non-null  object        
 1   timestamp    54750 non-null  datetime64[ns]
 2   temperature  54750 non-null  float64       
 3   season       54750 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 1.7+ MB


# Этап 1. Анализ исторических данных:

✅ Вычислить скользящее среднее и std температуры с окном в 30 дней для сглаживания краткосрочных колебаний.

✅ Выявить аномалии, где температура выходит за пределы  среднее ± 2𝜎 (используя скользящие значения).

✅ Рассчитать среднюю температуру и стандартное отклонение для каждого
сезона в каждом городе (Профиль сезона).

✅ Попробуйте распараллелить проведение этого анализа. Сравните скорость выполнения анализа с распараллеливанием и без него.

>учитывая сообщения в тг курса план анализа был немного скорректирован

### 1.1. Общий скрипт для анализа

In [4]:
# Cкользящие среднее и std температуры с окном в 30 дней

df['rolling_mean'] = df.groupby(['city', 'season'])['temperature'].transform(lambda x: x.rolling(window=30, min_periods=1).mean())
df['rolling_std'] = df.groupby(['city', 'season'])['temperature'].transform(lambda x: x.rolling(window=30, min_periods=1).std())

In [5]:
# Аномалии, где температура выходит за пределы среднее ± 2𝜎 (используя скользящие значения)

df['anomaly'] = df.apply(lambda x: True if(x["temperature"]>x['rolling_mean'] + 2*x['rolling_std']) | (x["temperature"]<x['rolling_mean'] - 2*x['rolling_std']) else False, axis=1)

In [6]:
# Профиль сезона

df['mean'] = df.groupby(['city', 'season'])['temperature'].transform('mean')
df['std'] = df.groupby(['city', 'season'])['temperature'].transform('std')

In [7]:
# Средняя температура по сезону и городу
df['mean_temperature'] = df.groupby(['city', 'season'])['temperature'].transform('mean')

# Стандартное отклонение температуры по сезону и городу
df['std_temperature'] = df.groupby(['city', 'season'])['temperature'].transform('std')

### 1.2. Функция анализа временных рядов для города

In [8]:
def analyze_city_temperature(args):

    df, city = args

    city_df = df[df['city'] == city].copy()

    # Cкользящие среднее и std температуры с окном в 30 дней
    city_df['rolling_mean'] = city_df.groupby(['season'])['temperature'].transform(lambda x: x.rolling(window=30, min_periods=1).mean())
    city_df['rolling_std'] = city_df.groupby(['season'])['temperature'].transform(lambda x: x.rolling(window=30, min_periods=1).std())

    # Аномалии, где температура выходит за пределы среднее ± 2𝜎 (используя скользящие значения)
    city_df['anomaly'] = city_df.apply(
        lambda x: True if(x["temperature"]>x['rolling_mean'] + 2*x['rolling_std']) or
                       (x["temperature"]<x['rolling_mean'] - 2*x['rolling_std']) else False, axis=1
        )

    # Профиль сезона
    season_profile = city_df.groupby('season')['temperature'].agg(['mean', 'std'])

    # Средняя, min, max температура за всё время
    avg_temp = city_df['temperature'].mean()
    min_temp = city_df['temperature'].min()
    max_temp = city_df['temperature'].max()

    # Список аномальных точек
    anomalies = city_df[city_df['anomaly']]

    # Результат-словарь для города
    result = {
        'city': city,
        'average_temperature': avg_temp,
        'min_temperature': min_temp,
        'max_temperature': max_temp,
        'season_profile': season_profile,
        'rolling_mean': city_df[['timestamp', 'rolling_mean']],
        'rolling_std': city_df[['timestamp', 'rolling_std']],
        'anomalies': anomalies[['timestamp', 'temperature']]
    }

    return result

In [10]:
data = pd.read_csv('temperature_data.csv')
cities = data['city'].unique()

args = [(data, city) for city in cities]

In [18]:
import time

# Последовательное выполнение
start_time = time.time()

results_sequential = [analyze_city_temperature((data, city)) for city in cities]

end_time = time.time()
print(f"Время выполнения (последовательно): {end_time - start_time:.2f} секунд")

Время выполнения (последовательно): 1.25 секунд


In [12]:
results_dict = {result['city']: result for result in results_sequential}

In [19]:
from multiprocessing import Pool, cpu_count

# Параллельное выполнение
start_time = time.time()

with Pool(processes=4) as pool:
    results_parallel = pool.map(analyze_city_temperature, args)

end_time = time.time()
print(f"Время выполнения (параллельно): {end_time - start_time:.2f} секунд")

Время выполнения (параллельно): 1.74 секунд


> При исследовании выяснила, что время выполнения параллельного исполнения не всегда меньше, чем при последовательном (что, как мне казалось, ожидалось). Это возможно связано с накладками на выделения ресурсов для распараллеливания, если бы данных было гораздо больше (* мои предположения и прочитанная информация) - эти накладки не так сильно ощущались.

# Этап 2. Мониторинг текущей температуры:
✅ Подключить OpenWeatherMap API для получения текущей температуры города.

✅ Получить текущую температуру для выбранного города через OpenWeatherMap API.

✅ Определить, является ли текущая температура нормальной, исходя из исторических данных для текущего сезона.

✅ Данные на самом деле не совсем реальные (сюрпрайз). Поэтому на момент эксперимента погода в Берлине, Каире и Дубае была в рамках нормы, а в Пекине и Москве аномальная. Протестируйте свое решение для разных городов.

✅ Попробуйте для получения текущей температуры использовать синхронные и асинхронные методы. Что здесь лучше использовать?

In [22]:
import requests
import datetime

# Синхронный метод

API_KEY = "Ваш API ключ"
BASE_URL = "http://api.openweathermap.org/data/2.5/weather"

def get_current_temperature(city):
    try:
        params = {
            "q": city,
            "appid": API_KEY,
            "units": "metric"
        }
        response = requests.get(BASE_URL, params=params)
        if response.status_code == 200:
            data = response.json()
            return data['main']['temp']
    except Exception as e:
        print("ERROR:", e)


start_time = time.time()

for city in cities:
  print(f"Температура в {city}: {get_current_temperature(city)}°C")

end_time = time.time()
print(f"Время выполнения (синхронный метод): {end_time - start_time:.2f} секунд")

Температура в New York: -1.49°C
Температура в London: 4.05°C
Температура в Paris: 7.29°C
Температура в Tokyo: 6.78°C
Температура в Moscow: -4.29°C
Температура в Sydney: 20.87°C
Температура в Berlin: 10.85°C
Температура в Beijing: 0.94°C
Температура в Rio de Janeiro: 28.11°C
Температура в Dubai: 19.96°C
Температура в Los Angeles: 20.83°C
Температура в Singapore: 25.68°C
Температура в Mumbai: 21.99°C
Температура в Cairo: 17.42°C
Температура в Mexico City: 19.97°C
Время выполнения (синхронный метод): 0.86 секунд


In [23]:
import aiohttp
import asyncio

# Асинхронный метод

async def get_current_temperature_async(city):
    params = {
        "q": city,
        "appid": API_KEY,
        "units": "metric"
    }

    try:
        async with aiohttp.ClientSession() as session:
            async with session.get(BASE_URL, params=params) as response:
                if response.status == 200:
                    data = await response.json()
                    return data['main']['temp']
    except Exception as e:
        print("ERROR:", e)

async def main():
    tasks = [get_current_temperature_async(city) for city in cities]
    temperatures = await asyncio.gather(*tasks)

    for city, temp in zip(cities, temperatures):
        if temp is not None:
            print(f"Температура в {city}: {temp}°C")

start_time = time.time()

await main()

end_time = time.time()
print(f"Время выполнения (асинхронный метод): {end_time - start_time:.2f} секунд")

Температура в New York: -1.49°C
Температура в London: 4.05°C
Температура в Paris: 7.29°C
Температура в Tokyo: 6.78°C
Температура в Moscow: -4.29°C
Температура в Sydney: 20.87°C
Температура в Berlin: 10.85°C
Температура в Beijing: 0.94°C
Температура в Rio de Janeiro: 28.11°C
Температура в Dubai: 19.96°C
Температура в Los Angeles: 20.83°C
Температура в Singapore: 25.68°C
Температура в Mumbai: 21.99°C
Температура в Cairo: 17.42°C
Температура в Mexico City: 19.97°C
Время выполнения (асинхронный метод): 0.07 секунд


> Асинхронный метод показал гораздо более быстрый результат выполнения запроса по сравнению с синхронным.

### Функционал проверки нормальности текущей температуры

In [24]:
def is_temperature_normal(current_temp, city, result_hist, current_season):
    city_result = result_hist.get(city)
    season_profile = city_result['season_profile']
    current_season_profile = season_profile.loc[season_profile.index == current_season]

    mean_temp = current_season_profile['mean'].values[0]
    std_temp = current_season_profile['std'].values[0]

    lower_bound = mean_temp - 2 * std_temp
    upper_bound = mean_temp + 2 * std_temp

    is_normal = lower_bound <= current_temp <= upper_bound
    explanation = f"Текущая температура {current_temp}°C {'входит в норму' if is_normal else 'выходит за пределы нормы'} ({lower_bound:.2f}°C до {upper_bound:.2f}°C)"

    return explanation

In [25]:
current_season = month_to_season[datetime.datetime.now().month]

for city in cities:
    current_temp = get_current_temperature(city)
    if current_temp is not None:
        result = is_temperature_normal(current_temp, city, results_dict, current_season)
        print(f"{result} для {city}")

Текущая температура -1.49°C входит в норму (-10.41°C до 10.51°C) для New York
Текущая температура 4.06°C входит в норму (-5.02°C до 14.79°C) для London
Текущая температура 7.29°C входит в норму (-5.35°C до 13.85°C) для Paris
Текущая температура 6.78°C входит в норму (-3.94°C до 16.23°C) для Tokyo
Текущая температура -4.29°C входит в норму (-20.26°C до -0.18°C) для Moscow
Текущая температура 20.86°C входит в норму (2.45°C до 21.62°C) для Sydney
Текущая температура 11.01°C выходит за пределы нормы (-10.00°C до 9.79°C) для Berlin
Текущая температура 0.94°C входит в норму (-11.86°C до 8.31°C) для Beijing
Текущая температура 28.11°C входит в норму (10.03°C до 29.86°C) для Rio de Janeiro
Текущая температура 19.96°C входит в норму (10.30°C до 29.75°C) для Dubai
Текущая температура 20.89°C входит в норму (5.04°C до 25.03°C) для Los Angeles
Текущая температура 25.68°C входит в норму (17.03°C до 37.13°C) для Singapore
Текущая температура 21.99°C входит в норму (15.35°C до 34.87°C) для Mumbai
Тек